# ERCOT Data Exploration

Exploratory data analysis of the ERCOT hourly load dataset used in the benchmark.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from energy_benchmark.data import ERCOTLoader
from energy_benchmark.data.preprocessing import preprocess_series

loader = ERCOTLoader(years=[2020, 2021, 2022, 2023, 2024])
series = loader.load()
series = preprocess_series(series)

print(f"Total observations: {len(series):,}")
print(f"Date range: {series.index.min()} — {series.index.max()}")
series.describe()

## Time Series Overview

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
series.plot(ax=ax, linewidth=0.3, alpha=0.8)
ax.set_ylabel("Load (MW)")
ax.set_title("ERCOT Total System Load (2020–2024)")
plt.tight_layout()
plt.show()

print("\n--- Interpretation ---")
print(f"The series spans {(series.index.max() - series.index.min()).days / 365:.1f} years "
      f"with {len(series):,} hourly observations.")
print(f"Load ranges from {series.min():,.0f} MW to {series.max():,.0f} MW "
      f"(mean: {series.mean():,.0f} MW).")
print("The strong annual cycle is clearly visible: Texas summers drive "
      "peak demand due to air conditioning, while winter shows a secondary "
      "peak from heating loads (especially visible after the Feb 2021 storm).")

## Seasonal Patterns

In [ ]:
# Average daily profile
hourly_avg = series.groupby(series.index.hour).mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(hourly_avg.index, hourly_avg.values, "o-")
axes[0].set_xlabel("Hour of day")
axes[0].set_ylabel("Mean load (MW)")
axes[0].set_title("Average Daily Load Profile")

# Monthly averages
monthly_avg = series.groupby(series.index.month).mean()
axes[1].bar(monthly_avg.index, monthly_avg.values)
axes[1].set_xlabel("Month")
axes[1].set_ylabel("Mean load (MW)")
axes[1].set_title("Average Monthly Load")

plt.tight_layout()
plt.show()

print("\n--- Interpretation ---")
peak_hour = hourly_avg.idxmax()
trough_hour = hourly_avg.idxmin()
peak_month = monthly_avg.idxmax()
print(f"Daily profile: load peaks at {peak_hour}:00 and dips at {trough_hour}:00.")
print(f"  The afternoon peak reflects commercial/residential cooling demand.")
print(f"  The overnight trough shows reduced activity and lower cooling needs.")
month_names = {1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",
               7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"}
print(f"Annual profile: highest mean load in {month_names[peak_month]} "
      f"({monthly_avg.max():,.0f} MW).")
print("  This strong summer peak is characteristic of hot-climate grids like Texas.")
print("  The seasonal amplitude (~15-20 GW) is a key challenge for forecasting models.")

## Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

series.hist(bins=80, ax=axes[0], edgecolor="white")
axes[0].set_xlabel("Load (MW)")
axes[0].set_title("Load Distribution")

# Year-over-year boxplot
df_box = series.to_frame("load")
df_box["year"] = df_box.index.year
sns.boxplot(data=df_box, x="year", y="load", ax=axes[1])
axes[1].set_title("Load Distribution by Year")
axes[1].set_ylabel("Load (MW)")

plt.tight_layout()
plt.show()

## Autocorrelation

In [ ]:
from pandas.plotting import autocorrelation_plot

fig, ax = plt.subplots(figsize=(14, 3))
# Use a subset for speed
autocorrelation_plot(series.iloc[:8760], ax=ax)  # 1 year
ax.set_xlim(0, 720)  # up to 30 days
ax.set_title("Autocorrelation (first year, up to 30 days)")
plt.tight_layout()
plt.show()

print("\n--- Interpretation ---")
print("Strong autocorrelation peaks at multiples of 24h (daily cycle) and 168h (weekly cycle).")
print("This confirms that a Seasonal Naive baseline with period=168 (weekly) is a natural choice.")
print("The slow decay of autocorrelation indicates long-range dependencies that")
print("foundation models — pre-trained on diverse time series — may capture better")
print("than local statistical models like ARIMA.")

## Train / Val / Test Split

In [ ]:
train, val, test = loader.split(series)

print(f"Train: {len(train):,} hrs ({train.index.min().date()} — {train.index.max().date()})")
print(f"Val:   {len(val):,} hrs ({val.index.min().date()} — {val.index.max().date()})")
print(f"Test:  {len(test):,} hrs ({test.index.min().date()} — {test.index.max().date()})")